# Librerías y funciones

## Librerías


In [2]:
import duckdb
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import polars as pl
import pandas as pd
import pyarrow
import reader
import time

from great_tables import GT
from math import ceil
from multiprocessing import Pool
from typing import Dict, Any
from typing import Optional

wd = "C:/Directorio_Trabajo/2024/IPC_Calc/"

ModuleNotFoundError: No module named 'duckdb'

## Funciones


In [2]:
# DataFrame de ponderaciones por agrupación y región desde hoja de Excel
def get_ponds_from_xlsx():
    df = pl.read_excel(
        source = wd + "Categorias.xlsx",
        sheet_name = "Regiones",
        infer_schema_length=None,
        schema_overrides={
            "División": pl.String,
            "Grupo": pl.String,
            "Clase": pl.String,
            "SubClase": pl.String,
            "Categoría": pl.String,
            "Producto": pl.String,
            },
    )[:,1:]
    df = df.with_columns(
        pl.col("Código").str.slice(0, 2).alias("División"),
        pl.col("Código").str.slice(0, 3).alias("Grupo"),
        pl.col("Código").str.slice(0, 4).alias("Clase"),
        pl.col("Código").str.slice(0, 5).alias("SubClase"),
        pl.col("Código").str.slice(0, 6).alias("Categoría"),
        pl.col("Código").str.slice(0, 8).alias("Producto"),
        )
    return df

# DataFrame de índices desde hoja de Excel
def get_df_from_xlsx(zona):
    df = pl.read_excel(
        source = wd + "Ejercicio calculo IPC - Investigación.xlsx",
        sheet_name = zona
    )[:,2:]
    return df

# Solo los datos que contienen Precio, Unidad de Medida o Contenido; depura establecimientos
def get_valid_columns_in_df(df):
    nombres_t = df.columns
    df[1,0]="Precio"
    df[1,1]="Unidad de Medida"
    df[1,2]="Contenido"

    check_list=["Precio","Unidad de Medida","Contenido"] 
    row_values = df.row(1)
    conditions_met = [value in check_list for value in row_values]
    df = df[conditions_met]
    return df

# Repetir los valores de los establecimientos en la primera fila
def repeat_names_in_row(row):
    row = list(row)
    for i in range(1, len(row)):
        if i % 3 != 0:
            row[i] = row[i - (i % 3)]
    return row

def replace_names_in_df(df,row_index,modified_row):
    row_values = df.row(row_index)
    df = df.with_columns([
        pl.when(
            pl.arange(0, df.height) == row_index
            ).then(
                pl.lit(modified_row[i])
            ).otherwise(pl.col(col)
            ).alias(col)
        for i, col in enumerate(df.columns)
    ])
    return df

def modify_df_names(df):
    modified_row = repeat_names_in_row(df.row(0))
    df = replace_names_in_df(df,0,modified_row)
    return df

def replace_unnamed(row):
    for i in range(1, len(row)):
        if r"UNNAMED" in row[i]:
            row[i] = row[i - 1]
    return row

def modify_df_establecimientos(df):
    modified_row = df.columns
    df = replace_names_in_df(df,2,modified_row)
    row_index = 2
    row_values = df.row(row_index)
    modified_row = replace_unnamed(list(row_values))
    df = replace_names_in_df(df,2,modified_row)
    return df

def obtener_dataframe(zona):
    df = get_df_from_xlsx(zona)
    df = get_valid_columns_in_df(df)
    df = modify_df_names(df)
    df = modify_df_establecimientos(df)
    df[0,1] = "."
    a_list = ["Unidad de Medida","Codigo",'.']
    df = df.filter(
        pl.col('Codigo').str.contains_any(a_list))
    indices_to_select = [1] + list(range(3, len(df.columns)))
    columns_to_select = [df.columns[i] for i in indices_to_select]
    df = df.select(columns_to_select)
    df = df.transpose(include_header=False)
    new_column_names_row = df.row(0)
    new_column_names = list(new_column_names_row)
    rename_dict = {
        old: new for old, new in zip(df.columns, new_column_names)}
    df = df.rename(rename_dict)
    df = df[1:,:]
    return df

def obtener_precio_t_1(zona):
    df = obtener_dataframe(zona)
    df_Precio_t_1 = df.filter(
        pl.col('Codigo').str.contains('t-1'))
    df_Precio_t_1 = df_Precio_t_1.filter(
        pl.col('Unidad de Medida').str.contains('Precio'))
    df_Precio_t_1.write_excel(
        workbook = wd + zona + "/" + zona + "_Precio_t_1.xlsx")
    word = ' Precio'
    pattern = f'{word}.*'
    df_Precio_t_1 = df_Precio_t_1.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    word = ' Precio'
    pattern = f'{word}.*'
    df_Precio_t_1 = df_Precio_t_1.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    return df_Precio_t_1

def obtener_contenido_t_1(zona):
    df = obtener_dataframe(zona)
    df_Contenido_t_1 = df.filter(
        pl.col('Codigo').str.contains('t-1'))
    df_Contenido_t_1 = df_Contenido_t_1.filter(
        pl.col('Unidad de Medida').str.contains('Contenido'))
    df_Contenido_t_1.write_excel(
        workbook = wd + zona + "/" + zona + "_Contenido_t_1.xlsx")
    word = ' Precio'
    pattern = f'{word}.*'
    df_Contenido_t_1 = df_Contenido_t_1.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    word = ' Precio'
    pattern = f'{word}.*'
    df_Contenido_t_1 = df_Contenido_t_1.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    return df_Contenido_t_1

def obtener_precio_t(zona):
    df = obtener_dataframe(zona)
    df_Precio_t = df.filter(
        ~pl.col('Codigo').str.contains('t-1'))
    df_Precio_t = df_Precio_t.filter(
        pl.col('Unidad de Medida').str.contains('Precio'))
    df_Precio_t.write_excel(workbook = wd + zona + "/" + zona + "_Precio_t.xlsx")
    word = ' Precio'
    pattern = f'{word}.*'
    df_Precio_t = df_Precio_t.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    return df_Precio_t

def obtener_contenido_t(zona):
    df = obtener_dataframe(zona)
    df_Contenido_t = df.filter(
        ~pl.col('Codigo').str.contains('t-1'))
    df_Contenido_t = df_Contenido_t.filter(
        pl.col('Unidad de Medida').str.contains('Contenido'))
    df_Contenido_t.write_excel(workbook = wd + zona + "/" + zona + "_Contenido_t.xlsx")
    word = ' Precio'
    pattern = f'{word}.*'
    df_Contenido_t = df_Contenido_t.with_columns(
        pl.col('Codigo').str.replace(pattern,""))
    return df_Contenido_t

In [3]:
def obtener_indices(wd,zona):
    # Valores en t
    p_t = obtener_precio_t(zona)
    my_columns = p_t.columns
    cols_to_process = my_columns[3:p_t.shape[1]]
    p_t[cols_to_process] = p_t[cols_to_process].cast(pl.Float64, strict=False)
    c_t = obtener_contenido_t(zona)
    my_columns = c_t.columns
    cols_to_process = my_columns[3:c_t.shape[1]]
    c_t[cols_to_process] = c_t[cols_to_process].cast(pl.Float64, strict=False)

    # Valores en t-1
    p_t_1 = obtener_precio_t_1(zona)
    my_columns = p_t_1.columns
    cols_to_process = my_columns[3:p_t_1.shape[1]]
    p_t_1[cols_to_process] = p_t_1[cols_to_process].cast(pl.Float64, strict=False)
    c_t_1 = obtener_contenido_t_1(zona)
    my_columns = c_t_1.columns
    cols_to_process = my_columns[3:c_t_1.shape[1]]
    c_t_1[cols_to_process] = c_t_1[cols_to_process].cast(pl.Float64, strict=False)

    # Índice de Precios por Establecimiento y Variedad
    i_t = p_t.clone()
    i_t[my_columns[3:i_t.shape[1]]] = (p_t[my_columns[3:c_t.shape[1]]] / c_t[my_columns[3:c_t.shape[1]]]) / (p_t_1[my_columns[3:c_t_1.shape[1]]] / c_t_1[my_columns[3:c_t_1.shape[1]]])
    i_t.write_excel(
        workbook = wd + zona + "/" + zona + "_Establecimiento.xlsx")

    # Índice de Precios por Variedad: Media Geométrica
    res_variedad = pl.DataFrame([
        pl.Series("Variedad", my_columns[3:i_t.shape[1]], dtype=pl.String)])
    res_variedad = res_variedad.with_columns(pl.col("Variedad").str.slice(0, 8).alias("Producto"))
    res_variedad = res_variedad.with_columns(
        Indice = 0.0)
    for row in range(res_variedad.shape[0]):
        res_variedad[row,2] = geometric_mean(
            i_t[res_variedad["Variedad"][row]].drop_nans() * 100)
    res_variedad.write_excel(
        workbook = wd + zona + "/" + zona + "_Variedad.xlsx")

    # Índice de Precios por Producto: Media Geométrica
    res_producto = (
        res_variedad.group_by("Producto", maintain_order=True)
        .agg(
            pl.map_groups(
                exprs=["Indice"],
                function=geometric_mean)
        ))
    res_producto = res_producto.join(
        ponderaciones_producto_region, 
        on="Producto")
    res_producto = res_producto.with_columns(
        (pl.col("Indice") * pl.col(zona) / 100).alias("Indice_Pond"),)
    res_producto[
        "Producto","Indice",zona,"Indice_Pond"].write_excel(
        workbook = wd + zona + "/" + zona + "_Producto.xlsx")

    ## Índice de Precios por Agrupaciones: Media Ponderada
    grupo = "Categoría"
    res_categoria = weighted_index_group_region(res_producto,grupo)
    res_categoria.write_excel(
        workbook = wd + zona +"/" + zona + "_" + grupo + ".xlsx")
    grupo = "SubClase"
    res_subclase = weighted_index_group_region(res_producto,grupo)
    res_subclase.write_excel(
        workbook = wd + zona +"/" + zona + "_" + grupo + ".xlsx")
    grupo = "Grupo"
    res_grupo = weighted_index_group_region(res_producto,grupo)
    res_grupo.write_excel(
        workbook = wd + zona +"/" + zona + "_" + grupo + ".xlsx")
    grupo = "División"
    res_division = weighted_index_group_region(res_producto,grupo)
    res_division.write_excel(
        workbook = wd + zona +"/" + zona + "_" + grupo + ".xlsx")

    # Resultados en DataFrames
    return p_t, c_t, p_t_1, c_t_1, i_t, res_variedad, res_producto, res_categoria, res_subclase, res_grupo, res_division

# Function to calculate geometric mean
def geometric_mean(series):
    return np.exp(np.log(series).mean())

def weighted_index_group_region(df,grupo):
    result = df.group_by(grupo).agg(
        [
            (pl.col("Indice") * pl.col(zona) / 100).sum(
            ).alias("weighted_sum"),
            pl.col(zona).sum(
            ).alias("Peso_" + grupo)
        ]).with_columns([
            (pl.col("weighted_sum") / pl.col("Peso_" + grupo) * 100
            ).alias("Índice_" + grupo)
        ]).select([grupo, "Peso_" + grupo, "Índice_" + grupo
        ]).sort(grupo)
    result = result.with_columns(
        (pl.col("Índice_" + grupo) * pl.col("Peso_" + grupo) / 100).alias("Indice_Pond"),
    )
    return result

# Índices por Zonas

- MDC = Metropolitana Distrito Central
- RUC = Resto Urbano Central
- MSPS =  Metropolitana San Pedro Sula
- RUN = Resto Urbano Norte
- ULA = Urbana Litoral Atlántico
- UOri = Urbana Oriental
- UOcc = Urbana Occidental
- US = Urbana Sur

## Ponderaciones

Para obtener el IPC, se tienen ponderaciones por producto, para cada zona:


In [4]:
zonas = ["MDC","RUC","MSPS","RUN","ULA","UOri","UOcc","US"]
ponderaciones_producto_region = get_ponds_from_xlsx()
GT(ponderaciones_producto_region[0:5,:]
    ).fmt_number(columns=zonas, decimals=4)

Código,CCIF,División,Grupo,Clase,SubClase,Categoría,Producto,MDC,RUC,MSPS,RUN,ULA,UOri,UOcc,US
01111201,Arroz clasificado,01,011,0111,01111,011112,01111201,0.5700,1.4588,0.3509,0.4085,0.5434,0.4632,0.3412,0.4629
01111601,Maíz en grano o desgranado,01,011,0111,01111,011116,01111601,0.4200,0.8953,1.3982,0.5928,0.9992,0.8363,0.8863,1.4378
01112101,Harina de trigo,01,011,0111,01112,011121,01112101,0.0400,0.3975,0.3677,0.2095,0.3380,0.5078,0.3144,0.7109
01112601,Harina de maíz,01,011,0111,01112,011126,01112601,0.0500,0.9162,1.6122,0.7486,1.5863,1.1931,1.7767,0.9332
01113101,Bollito de yema pan dulce,01,011,0111,01113,011131,01113101,0.0100,0.0834,0.1171,0.0745,0.1325,0.0712,0.0870,0.0534


Se agregó en el archivo de Excel, además de las ponderaciones por zona y producto (columnas), lo siguiente:

- Código: correspondiente al CCIF;
- CCIF: nombre del producto;
- División: los primeros dos dígitos del Código;
- Grupo: los primeros tres dígitos del Código;
- Clase: los primeros cuatro dígitos del Código;
- SubClase: los primeros cinco dígitos del Código;
- Categoría: los primeros seis dígitos del Código;
- Producto: correspondiente al CCIF.

## Cálculos por Zona

Se realizan mediante una función, que agrega archivos a las carpetas nombradas con las siglas señaladas por zona. 

La función crea dataframes que podrían utilizarse para visualizar los cálculos contenidos en los archivos; los nombres de los dataframes y archivos generados (incluyendo la sigla inicial de la zona) son los siguientes:

Cada archivo tiene un nombre que inicia también con las siglas de la región correspondiente. 

Para detallar cada dataframe y archivo de Excel resultante, se usará como ejemplo los resultados de la zona MDC:


In [5]:
zona = "MDC"
p_t, c_t, p_t_1, c_t_1, i_t, res_variedad, res_producto, res_categoria, res_subclase, res_grupo, res_division = obtener_indices(wd,zona)

### 1. Precios y Contenido en t, t-1

- p_t, p_t_1, Zona_Precio_t.xlsx, Zona_Precio_t_1.xlsx = precios en t y t-1, por establecimiento y variedad


In [6]:
GT(p_t[0:5,0:5])

.,Unidad de Medida,Codigo,01111201.01,01111201.02
Yip's,Precio,Supermercado,55.1096,NaN
La Colonia # 1,Precio,Supermercado,58.40515408,NaN
Pronto,Precio,Tienda de conveniencia,NaN,NaN
Foodmart,Precio,Tienda de conveniencia,NaN,NaN
Los almendros,Precio,Mercadito,NaN,12.2705


In [7]:
GT(p_t_1[0:5,0:5])

.,Unidad de Medida,Codigo,01111201.01,01111201.02
Yip's,Precio,Supermercado,52.0,NaN
La Colonia # 1,Precio,Supermercado,59.8,NaN
Pronto,Precio,Tienda de Conveniencia,NaN,NaN
Foodmart,Precio,Tienda de Conveniencia,NaN,NaN
Los almendros,Precio,Mercadito,NaN,11.0


- c_t, c_t_1, Zona_Contenido_t.xlsx, Zona_Contenido_t_1.xlsx = contenido (en unidades de medida correspondientes a cada producto) en t y t-1, por establecimiento y variedad.


In [8]:
GT(c_t[0:5,0:5])

.,Unidad de Medida,Codigo,01111201.01,01111201.02
Yip's,Contenido,Supermercado,1.75,NaN
La Colonia # 1,Contenido,Supermercado,1.75,NaN
Pronto,Contenido,Tienda de conveniencia,NaN,NaN
Foodmart,Contenido,Tienda de conveniencia,NaN,NaN
Los almendros,Contenido,Mercadito,NaN,1.0


In [9]:
GT(c_t_1[0:5,0:5])

.,Unidad de Medida,Codigo,01111201.01,01111201.02
Yip's,Contenido,Supermercado,1.75,NaN
La Colonia # 1,Contenido,Supermercado,1.75,NaN
Pronto,Contenido,Tienda de Conveniencia,NaN,NaN
Foodmart,Contenido,Tienda de Conveniencia,NaN,NaN
Los almendros,Contenido,Mercadito,NaN,1.0


### 2. Índice de Precios por Establecimiento y Variedad

- i_{e_v} = índice de precios unitario (por unidad de medida), por establecimiento y variedad

$i_{ev}^r = \frac{\frac{p_{ev,t}^r}{c_{ev,t}^r}}{\frac{p_{ev,t-1}^r}{c_{ev,t-1}^r}}$


In [10]:
GT(i_t[0:5,0:5])

.,Unidad de Medida,Codigo,01111201.01,01111201.02
Yip's,Precio,Supermercado,1.0597999999999999,NaN
La Colonia # 1,Precio,Supermercado,0.9766748173913045,NaN
Pronto,Precio,Tienda de conveniencia,NaN,NaN
Foodmart,Precio,Tienda de conveniencia,NaN,NaN
Los almendros,Precio,Mercadito,NaN,1.1155


### 3. Índice de Precios por Variedad: Media Geométrica

Como en la tabla mostrada anteriormente, cada columna corresponde a los índices por variedad, el índice a calcular es la media geométrica por columna:

$i_V^r = \sqrt[n]{{i_{ev_1}^r,i_{ev_2}^r,...,i_{ev_n}^r}} \text{ for v in } V^r$

$i_V^r = \exp (\frac{{\ln i_{ev_1}^r + \ln i_{ev_2}^r +...+\ln i_{ev_n}^r}}{n}) \text{ for v in } V^r$

Los resultados se guardan de manera que cada fila coresponde al índice calculado por variedad.


In [11]:
GT(res_variedad[0:5,:])

Variedad,Producto,Indice
01111201.01,01111201,101.73888005434813
01111201.02,01111201,111.27466018820274
01111201.03,01111201,124.85321521214229
01111601.01,01111601,106.14989401784626
01112101.02,01112101,117.42031436986977


### 4. Índice de Precios por Producto (X): Media Geométrica

$i_X^r = \sqrt[n]{{i_{V_1}^r,i_{V_2}^r,...,i_{V_n}^r}} \text{ for V in } X^r$

$i_X^r = \exp (\frac{{\ln i_{V_1}^r + \ln i_{V_2}^r +...+\ln i_{V_n}^r}}{n}) \text{ for V in } X^r$


In [12]:
GT(res_producto[0:5,["Producto","Indice",zona,"Indice_Pond"]])

Producto,Indice,MDC,Indice_Pond
01111201,112.2262160580291,0.57,0.6396894315307659
01111601,106.14989401784626,0.42,0.44582955487495435
01112101,117.42031436986977,0.04,0.04696812574794791
01112601,140.99028849208457,0.05,0.0704951442460423
01113101,112.80584502798534,0.01,0.011280584502798533


En vista que cada producto tiene una ponderación diferente por región (w_R), se calcula una columna que contiene el índice ponderado por producto:

$Indice\_Pond^r$ = $i_X^r * w_X^r$

La suma de esta columna corresponde al índice de precios al consumidor de la región:

$IPC^r$ = $\sum{Indice\_Pond^r}$


In [13]:
# print(res_producto[zona].sum())
print(res_producto["Indice_Pond"].sum())

114.15366197044139


### 5. Índice de Precios por Agrupaciones (A): Media Ponderada

El peso de cada agrupación dentro del IPC regional corresponde a la suma de los pesos regionales de los productos correspondientes a dicha agrupación:

$w_A^r = \frac{w_X^r}{\sum w_X^r} \text{ for X in } A^r$

El índice por agrupación es igual a:

$i_A^r = \sum i_X^r * \frac{w_X^r}{w_A^r} \text{ for X in } A^r$

Al igual que en los índices por producto, se agrega la columna Indice_Pond:

$Indice\_Pond^r$ = $i_A^r * w_A^r$

La suma de esta columna corresponde al índice de precios al consumidor de la región:

$IPC^r$ = $\sum{Indice\_Pond^r}$

#### Categoría


In [14]:
print(res_categoria["Indice_Pond"].sum())
GT(res_categoria[0:5,:])

114.1536619704414


Categoría,Peso_Categoría,Índice_Categoría,Indice_Pond
011112,0.57,112.22621605802911,0.6396894315307659
011116,0.42,106.14989401784626,0.44582955487495435
011121,0.04,117.42031436986977,0.04696812574794791
011126,0.05,140.99028849208457,0.0704951442460423
011131,0.942,111.96627568727173,1.0547223169740998


#### SubClase


In [15]:
print(res_subclase["Indice_Pond"].sum())
GT(res_subclase[0:10,:])

114.1536619704414


SubClase,Peso_SubClase,Índice_SubClase,Indice_Pond
01111,0.99,109.64838246522426,1.0855189864057202
01112,0.09,130.5147444377669,0.1174632699939902
01113,1.852,118.56707220785758,2.1958621772895226
01114,1.23,104.87399520785875,1.2899501410566625
01115,0.237,124.35390499426882,0.2947187548364171
01119,0.9470000000000001,122.92978910753418,1.1641451028483487
01122,4.093,107.92018226376776,4.417173060056014
01124,0.053,104.66199245066372,0.05547085599885177
01125,2.014,110.7649051063508,2.230805188841905
01131,0.294,113.22886594475581,0.33289286587758204


#### Grupo


In [16]:
print(res_grupo["Indice_Pond"].sum())
GT(res_grupo)

114.15366197044139


Grupo,Peso_Grupo,Índice_Grupo,Indice_Pond
011,38.603,114.88915746256303,44.35066145527321
012,3.62,124.16452256502156,4.49475571685378
031,11.500000000000002,103.8877501041775,11.947091261980416
032,4.669999999999999,109.88495843954829,5.131627559126904
051,2.19,112.90277987057502,2.472570879165593
052,1.84,113.70705870184126,2.0922098801138795
053,3.08,111.17810052456048,3.424285496156463
054,3.0999999999999996,120.89531204629482,3.7477546734351392
055,2.09,124.24306522818671,2.596680063269102
056,5.619999999999998,127.77149205484541,7.1807578534823095


#### División


In [17]:
print(res_division["Indice_Pond"].sum())
GT(res_division)

114.15366197044138


División,Peso_División,Índice_División,Indice_Pond
01,42.223000000000006,115.68438332692364,48.84541717212697
03,16.17,105.61978244345897,17.07871882110732
05,17.919999999999998,120.0572480224469,21.514258845622482
06,23.68700000000001,112.78451104650063,26.715267131584614


# IPC, Ponderado

Teniendo calculado el índice de precios por región, el IPC agregado se obtiene asignando una ponderación los índices para cada zona:


In [18]:
ponderaciones_zona = pl.read_excel(
    source = wd + "Categorias.xlsx",
    sheet_name = "Zonas",
    infer_schema_length=None,
)

El proceso se tarda aproximadamente dos minutos.


In [19]:
# Para ejecutar todas las zonas:
zonas = ["MDC","RUC","MSPS","RUN","ULA","UOri","UOcc","US"]
for i, zona in enumerate(zonas):
    res = obtener_indices(wd,zona)
    ponderaciones_zona[i,3] = res[10]["Indice_Pond"].sum()
GT(ponderaciones_zona)

Ponderación,División,Índice Periodo t-1 (Periodo Base),Índice Periodo t
0.34,MDC,100,114.15366197044138
0.05,RUC,100,114.42901788035363
0.31,MSPS,100,113.97043474644593
0.1,RUN,100,113.46251395753553
0.07,ULA,100,115.05904156929545
0.03,UOri,100,114.32015557783441
0.06,Uocc,100,114.20476415578052
0.04,US,100,114.72056723216507


In [20]:
ipc = sum(ponderaciones_zona["Ponderación"] * ponderaciones_zona["Índice Periodo t"])
print("IPC General =" +  str(ipc))

IPC General =114.1356282469387
